# Экспорт и импорт данных

1. Подготовить данные для построения модели.
2. Сохранить готовые данные в: CSV, HDF5

In [1]:
import pandas as pd
import numpy as np

from os import getcwd
from os.path import getsize
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from core.reduce_mem_usage import reduce_mem_usage

## Загрузка данных

In [2]:
buildings = pd.read_csv("../data/buildings.csv")
weather = pd.read_csv("../data/weather.csv")
energy = pd.read_csv("../data/train.0.zip")

## Взять 20 зданий, объединить, оптимизировать

In [3]:
weather = weather[weather["site_id"] == 0]
energy = energy[energy["building_id"] < 20]
energy = pd.merge(
    left=energy,
    right=buildings,
    how="left",
    left_on="building_id",
    right_on="building_id",
)
del buildings

## Интерполяция значений

In [4]:
weather["precip_depth_1_hr"] = weather["precip_depth_1_hr"].apply(lambda x: x if x> 0 else 0)
interpolate_columns = [
    "air_temperature",
    "dew_temperature",
    "cloud_coverage",
    "wind_speed",
    "wind_direction",
    "precip_depth_1_hr",
    "sea_level_pressure",
]

for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction="both", kind="cubic")

## Обогащение данных: погода

In [5]:
weather["air_temperature_diff_1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff_1"] = weather.at[1, "air_temperature_diff_1"]
weather["air_temperature_diff_2"] = weather["air_temperature_diff_1"].diff()
weather.at[0, "air_temperature_diff_2"] = weather.at[1, "air_temperature_diff_2"]

## Объединение погодных данных

In [6]:
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(
    left=energy,
    right=weather,
    how="left",
    left_index=True,
    right_index=True,
)
energy.reset_index(inplace=True)
energy = energy.drop(
    columns=[
        "meter",
        "site_id",
        "year_built",
        "square_feet",
        "floor_count",
    ], 
    axis=1
)
del weather
energy = reduce_mem_usage(energy)
print(energy.info())

Потребление памяти меньше на - 12.4 Мб (минус 71.2%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   timestamp               175680 non-null  datetime64[ns]
 1   building_id             175680 non-null  int8          
 2   meter_reading           175680 non-null  float16       
 3   primary_use             175680 non-null  category      
 4   air_temperature         175680 non-null  float16       
 5   cloud_coverage          175680 non-null  float16       
 6   dew_temperature         175680 non-null  float16       
 7   precip_depth_1_hr       175680 non-null  float16       
 8   sea_level_pressure      175680 non-null  float16       
 9   wind_direction          175680 non-null  float16       
 10  wind_speed              175680 non-null  float16       
 11  air_temperature_diff_1  175680 non-nul

## Обогащение данных: дата

In [7]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
energy["week"] = energy["timestamp"].dt.isocalendar().week.astype("int8")
energy["month"] = energy["timestamp"].dt.month.astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)

dates_range = pd.date_range(start="2015-12-31", end="2017-01-01")
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
energy["is_holiday"] = energy["date"].isin(us_holidays).astype("int8")

for weekday in range(0, 7):
    energy[f"is_wday {str(weekday)}"] = energy["weekday"].isin([weekday]).astype("int8")
for week in range(1, 54):
    energy[f"is_w {str(week)}"] = energy["week"].isin([week]).astype("int8")
for month in range(1, 13):
    energy[f"is_m {str(month)}"] = energy["month"].isin([month]).astype("int8")

## Логарифмирование данных

In [8]:
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

## Экспорт в CSV

In [9]:
print(energy.info())
energy.to_csv("../data/out/energy.0-20.csv.gz", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 92 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   timestamp               175680 non-null  datetime64[ns]
 1   building_id             175680 non-null  int8          
 2   meter_reading           175680 non-null  float16       
 3   primary_use             175680 non-null  category      
 4   air_temperature         175680 non-null  float16       
 5   cloud_coverage          175680 non-null  float16       
 6   dew_temperature         175680 non-null  float16       
 7   precip_depth_1_hr       175680 non-null  float16       
 8   sea_level_pressure      175680 non-null  float16       
 9   wind_direction          175680 non-null  float16       
 10  wind_speed              175680 non-null  float16       
 11  air_temperature_diff_1  175680 non-null  float16       
 12  air_temperature_diff_2  175680

In [10]:
energy = pd.read_csv("../data/out/energy.0-20.csv.gz")
print(energy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 92 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   timestamp               175680 non-null  object 
 1   building_id             175680 non-null  int64  
 2   meter_reading           175680 non-null  float64
 3   primary_use             175680 non-null  object 
 4   air_temperature         175680 non-null  float64
 5   cloud_coverage          175680 non-null  float64
 6   dew_temperature         175680 non-null  float64
 7   precip_depth_1_hr       175680 non-null  float64
 8   sea_level_pressure      175680 non-null  float64
 9   wind_direction          175680 non-null  float64
 10  wind_speed              175680 non-null  float64
 11  air_temperature_diff_1  175680 non-null  float64
 12  air_temperature_diff_2  175680 non-null  float64
 13  hour                    175680 non-null  int64  
 14  weekday             

## Экспорт в HDF5

In [11]:
energy = reduce_mem_usage(energy)
energy.to_hdf(
    "../data/out/energy.0-20.h5",
    key="energy",
    format="table",
    complib="zlib",
    complevel=9,
    mode="w"
)


Потребление памяти меньше на - 104.7 Мб (минус 84.9%)


In [12]:
print(f"CSV: {getsize('../data/out/energy.0-20.csv.gz')}")
print(f"HDF5: {getsize('../data/out/energy.0-20.h5')}")

CSV: 1529687
HDF5: 861608


In [13]:
energy = pd.read_hdf("../data/out/energy.0-20.h5", "energy")
print(energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 175679
Data columns (total 92 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   timestamp               175680 non-null  datetime64[ns]
 1   building_id             175680 non-null  int8          
 2   meter_reading           175680 non-null  float16       
 3   primary_use             175680 non-null  category      
 4   air_temperature         175680 non-null  float16       
 5   cloud_coverage          175680 non-null  float16       
 6   dew_temperature         175680 non-null  float16       
 7   precip_depth_1_hr       175680 non-null  float16       
 8   sea_level_pressure      175680 non-null  float16       
 9   wind_direction          175680 non-null  float16       
 10  wind_speed              175680 non-null  float16       
 11  air_temperature_diff_1  175680 non-null  float16       
 12  air_temperature_diff_2  175680

## Разделение данных

In [14]:
energy_train, energy_test = train_test_split(energy[energy["meter_reading"] > 0], test_size=0.2)
print(energy_train.head())

                 timestamp  building_id  meter_reading          primary_use  \
109797 2016-08-16 17:00:00           17      68.375000               Office   
94205  2016-07-15 06:00:00            5      29.015625            Education   
127591 2016-09-22 19:00:00           11     497.000000            Education   
147412 2016-11-03 02:00:00           12     258.250000  Lodging/residential   
128661 2016-09-25 01:00:00            1     136.250000            Education   

        air_temperature  cloud_coverage  dew_temperature  precip_depth_1_hr  \
109797        30.000000        6.667969        24.000000               36.0   
94205         25.000000        2.000000        22.796875                0.0   
127591        31.703125        4.000000        21.703125                0.0   
147412        21.703125        4.667969        17.796875                0.0   
128661        26.703125        6.000000        23.296875                0.0   

        sea_level_pressure  wind_direction  ...  i

In [15]:
pd.set_option("io.hdf.default_format", "table")
store = pd.HDFStore("../data/out/energy.0-20.split.h5",
                    complevel=9,
                    complib="zlib",
                    mode="w",
                    )
store["energy_train"] = energy_train
store["energy_test"] = energy_test
store.put("metadata",
          pd.Series(["Набор обогащенных тестовых данных по 20 зданиям"]))
store.close()

In [16]:
print(f"HDF5: {getsize('../data/out/energy.0-20.split.h5')}")

HDF5: 3878532


## Чтение из HDF5

In [17]:
store = pd.HDFStore("../data/out/energy.0-20.split.h5")
energy_test = store.get("energy_test")[:]
energy_train = store.get("energy_train")[:]
metadata = store.get("metadata")[:]
store.close()
print(metadata[0])
print(energy_train.head())

Набор обогащенных тестовых данных по 20 зданиям
                 timestamp  building_id  meter_reading          primary_use  \
109797 2016-08-16 17:00:00           17      68.375000               Office   
94205  2016-07-15 06:00:00            5      29.015625            Education   
127591 2016-09-22 19:00:00           11     497.000000            Education   
147412 2016-11-03 02:00:00           12     258.250000  Lodging/residential   
128661 2016-09-25 01:00:00            1     136.250000            Education   

        air_temperature  cloud_coverage  dew_temperature  precip_depth_1_hr  \
109797        30.000000        6.667969        24.000000               36.0   
94205         25.000000        2.000000        22.796875                0.0   
127591        31.703125        4.000000        21.703125                0.0   
147412        21.703125        4.667969        17.796875                0.0   
128661        26.703125        6.000000        23.296875                0.0   

  